In [1]:
# activate project environment
# include these lines of code in any future scripts/notebooks
#---
import Pkg
if !haskey(Pkg.installed(), "AA228FinalProject")
    jenv = joinpath(dirname(@__FILE__()), ".") # this assumes the notebook is in the same dir
    # as the Project.toml file, which should be in top level dir of the project. 
    # Change accordingly if this is not the case.
    Pkg.activate(jenv)
end
Pkg.instantiate()
Pkg.build("Cairo")
#---

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Fetching: [========================================>]  100.0 %.0 %>     ]  85.1 %  Updating registry at `~/.julia/registries/JuliaPOMDP`
  Updating git-repo `https://github.com/JuliaPOMDP/Registry`
  Building LibCURL ─→ `~/.julia/packages/LibCURL/OoXMv/deps/build.log`
  Building WinRPM ──→ `~/.julia/packages/WinRPM/Y9QdZ/deps/build.log`
  Building Homebrew → `~/.julia/packages/Homebrew/l8kUw/deps/build.log`
  Building Cairo ───→ `~/.julia/packages/Cairo/CXPG1/deps/build.log`


In [3]:
# import necessary packages
Pkg.add("JLD")
Pkg.add("CPUTime")
using JLD
using AA228FinalProject
using POMDPPolicies
using BeliefUpdaters
using ParticleFilters
using POMDPSimulators
using Cairo
using Gtk
using Random
using Printf
using CPUTime

 Resolving package versions...
  Updating `Project.toml`
 [no changes]
  Updating `Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `Project.toml`
  [a9c8d775] + CPUTime v0.1.0
  Updating `Manifest.toml`
 [no changes]


┌ Info: Precompiling CPUTime [a9c8d775-2e2e-55fc-8582-045d282d599e]
└ @ Base loading.jl:1186
┌ Warning: Module Compat with build ID 123967591500933 is missing from the cache.
│ This may mean Compat [34da2185-b29b-5c13-b0c7-acf172513d20] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:941


In [5]:
@load "fib_policy.jld"
@load "qmdp_policy.jld"

1-element Array{Symbol,1}:
 :qmdp_policy

In [9]:
sensor = Bumper()
config = 3 # 1,2, or 3
ds = DiscreteRoombaStateSpace(50, 50, 20)
v_steps = [2,5]
om_steps = [-3*pi/4, -pi/2, -pi/4, 0.0, pi/4, pi/2, 3*pi/4, pi]
aspace = vec(collect(RoombaAct(v,om) for v in v_steps, om in om_steps))
m = RoombaPOMDP(sensor=sensor, mdp=RoombaMDP(sspace=ds, aspace=aspace, config=config, contact_pen=-0.5))

num_particles = 5000
resampler = BumperResampler(num_particles)

spf = SimpleParticleFilter(m, resampler)

v_noise_coefficient = 2.0
om_noise_coefficient = 0.5

belief_updater = RoombaParticleFilter(spf, v_noise_coefficient, om_noise_coefficient);



In [10]:
using Statistics

total_rewards = []
avg_step_time = 0
num_experiments = 20

for exp = 1:num_experiments
    println(string(exp))
    
    Random.seed!(exp)
    
    traj_rewards = 0
    num_steps = 0
    CPUtic()
    for step in stepthrough(m, qmdp_policy, belief_updater, max_steps=100)
        num_steps += 1
        traj_rewards += step.r
    end
    time_elapsed = CPUtoq()
    avg_step_time += time_elapsed / num_steps
    
    push!(total_rewards, traj_rewards)
end
avg_step_time = avg_step_time / num_experiments
@printf("Average Step Time: %3f ", avg_step_time)
@printf("Mean Total Reward: %.3f, StdErr Total Reward: %.3f", mean(total_rewards), std(total_rewards)/sqrt(num_experiments))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
Average Step Time: 0.400419 Mean Total Reward: -3.630, StdErr Total Reward: 1.639

In [11]:
using Statistics

total_rewards = []
avg_step_time = 0
num_experiments = 20

for exp = 1:num_experiments
    println(string(exp))
    
    Random.seed!(exp)
    
    traj_rewards = 0
    num_steps = 0
    CPUtic()
    for step in stepthrough(m, fib_policy, belief_updater, max_steps=100)
        num_steps += 1
        traj_rewards += step.r
    end
    time_elapsed = CPUtoq()
    avg_step_time += time_elapsed / num_steps
    
    push!(total_rewards, traj_rewards)
end
avg_step_time = avg_step_time / num_experiments
@printf("Average Step Time: %3f ", avg_step_time)
@printf("Mean Total Reward: %.3f, StdErr Total Reward: %.3f", mean(total_rewards), std(total_rewards)/sqrt(num_experiments))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
Average Step Time: 0.407561 Mean Total Reward: 0.315, StdErr Total Reward: 1.873